# ISSS621 - Data Science for Business
# Project
## Making Recommendations of H&M products based on Market Basket Analysis
#### The Model :
We used Apriori algorithm for frequent pattern mining and planned to use association rule to obtain recommendations
#### Limitation :
- The analysis is limited by the memory and exectuion power of our machines which limits us to transaction data of the latest 3 months. This in turn limits the accuracy of our model as we do not get all the transactions of the customer over a longer period of time
- This method did not generate any frequen multi-itemsets mainly because the model is not approprite for fashion items as the basket size for fashion items is too small. A suitable application case is for grocery basket for which the basket size can easily reach 10 to 20 items

In [1]:
import pandas as pd
import numpy as np

In [2]:
# customer = pd.read_csv("customers.csv")
transaction_train = pd.read_csv("transactions_train.csv")

In [3]:
transaction_train.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [4]:
transaction_train.describe()

,article_id,price,sales_channel_id
count,3.178832e+07,3.178832e+07,3.178832e+07
mean,6.962272e+08,2.782927e-02,1.704028e+00
std,1.334480e+08,1.918113e-02,4.564786e-01
min,1.087750e+08,1.694915e-05,1.000000e+00
25%,6.328030e+08,1.581356e-02,1.000000e+00
50%,7.145820e+08,2.540678e-02,2.000000e+00
75%,7.865240e+08,3.388136e-02,2.000000e+00
max,9.562170e+08,5.915254e-01,2.000000e+00


In [5]:
transaction_train.isnull().sum()

t_dat               0
customer_id         0
article_id          0
price               0
sales_channel_id    0
dtype: int64

In [6]:
pd.options.mode.chained_assignment = None

In [7]:
transaction_train = transaction_train[['t_dat','customer_id','article_id']]

In [8]:
print(min(transaction_train['t_dat']),max(transaction_train['t_dat']))

2018-09-20 2020-09-22


In [9]:
df = transaction_train[transaction_train['t_dat']>='2020-07-01']

In [10]:
df.shape

(3386963, 3)

In [11]:
# assume items bought by the same customer on the same day is on the same receipt (basket)
df['dummy_invoice']=df.groupby(['t_dat','customer_id']).ngroup()

In [12]:
df.head(15)

,t_dat,customer_id,article_id,dummy_invoice
28401361,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,825577003,0
28401362,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,849487005,0
28401363,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,772659001,0
28401364,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,804732004,0
28401365,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,804732004,0
28401366,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,804732003,0
28401367,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,887949004,0
28401368,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,824194004,0
28401369,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,876461001,0
28401370,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,869796001,0


In [13]:
df['no_unique_article'] = df.groupby('dummy_invoice')['article_id'].transform('nunique')
df.head(15)

,t_dat,customer_id,article_id,dummy_invoice,no_unique_article
28401361,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,825577003,0,11
28401362,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,849487005,0,11
28401363,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,772659001,0,11
28401364,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,804732004,0,11
28401365,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,804732004,0,11
28401366,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,804732003,0,11
28401367,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,887949004,0,11
28401368,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,824194004,0,11
28401369,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,876461001,0,11
28401370,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,869796001,0,11


In [14]:
df.tail()

,t_dat,customer_id,article_id,dummy_invoice,no_unique_article
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,1040900,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,1040900,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,1040901,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,1040902,1
31788323,2020-09-22,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,898573003,1040903,1


In [15]:
basket_plus = df[df['no_unique_article']>1]
print(df.shape,basket_plus.shape)

(3386963, 5) (3002103, 5)


In [16]:
# cast to str due to an error from frequent itemset mining later
basket_plus.article_id = [str(i) for i in basket_plus.article_id]

In [17]:
# transform data to lists
transactions = basket_plus.groupby('dummy_invoice')['article_id'].apply(list)
transactions

dummy_invoice
0          [825577003, 849487005, 772659001, 804732004, 8...
2                          [559616013, 559616013, 566140001]
3                          [858172002, 524529004, 907696002]
4               [617787001, 564313022, 764147003, 854247002]
5                          [822311010, 843772002, 701472001]
                                 ...                        
1040896                               [895730002, 740519002]
1040897    [919273002, 881448002, 919273001, 919273004, 8...
1040898                               [918522001, 918485001]
1040899    [896169005, 791587021, 791587021, 910949002, 7...
1040900                               [929511001, 891322004]
Name: article_id, Length: 696233, dtype: object

doc:http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/#example-3-working-with-sparse-representations

In [18]:
# use sparse matrix
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
oht_ary = te.fit(transactions).transform(transactions, sparse=True)
basket_encode = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)
basket_encode

,108775015,108775044,110065001,110065002,110065011,111565001,111586001,111593001,111609001,120129001,...,948152002,949198001,949551001,949551002,949594001,952267001,952938001,953450001,953763001,956217002
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696228,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
696229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
696230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
696231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
from mlxtend.frequent_patterns import apriori

In [20]:
frequent_itemsets = apriori(basket_encode, min_support=0.002,
                                 use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)

frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

frequent_itemsets

,support,itemsets,length
0,0.005843,(751471001),1
1,0.005699,(706016001),1
2,0.005227,(372860002),1
3,0.004491,(610776002),1
4,0.004361,(730683050),1
...,...,...,...
75,0.002071,(915526002),1
76,0.002038,(865929003),1
77,0.002031,(907696002),1
78,0.002018,(711053003),1


In [21]:
frequent_itemsets[frequent_itemsets['length']>1]

,support,itemsets,length


In [22]:
# no multi-itemset, no need to proceed
from mlxtend.frequent_patterns import association_rules

In [23]:
association_rules(frequent_itemsets, metric ='lift',
                 min_threshold=1).sort_values('lift', ascending=False).reset_index(drop=True)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


#### confirm that basket size is indeed too small, which lead to failure of MBA

In [26]:
df_by_invoice=df.groupby(['dummy_invoice']).size()
df_by_invoice

dummy_invoice
0          12
1           1
2           3
3           3
4           4
           ..
1040899     9
1040900     2
1040901     1
1040902     1
1040903     1
Length: 1040904, dtype: int64

In [32]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [33]:
df_by_invoice.describe()

count   1040904.00000
mean          3.25387
std           3.17920
min           1.00000
25%           1.00000
50%           2.00000
75%           4.00000
max         178.00000
dtype: float64